In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
#import ipywidgets as ipw

import numpy as np
import pandas as pd
import datetime
import time
import json

import requests
import pydantic
import typing
import enum

import urllib
import importlib

ModuleNotFoundError: No module named 'requests'

In [ ]:
#import RMSBQL
import NotePublisher

In [ ]:
config_file_name = 'bql_rms_rest_api_config.json'
note_pub = NotePublisher.NotePublisher(config_file_name)

In [ ]:
url = note_pub.getURL()
uuid = note_pub.getUUID()
__headers = note_pub.getHeaders()
tokens = note_pub.getTokens()
__synchronizerToken = tokens['synchronizerToken']
__synchronizerUri = tokens['synchronizerUri']

In [ ]:
### UUID SHAREABLE DESCRIPTIONS
# 'BLOOMBERG/ 731 LEX'
# 'BLOOMBERG/ 919 3RD AVE'


### PEOPLE/AUTHOR DESCRIPTIONS

# 'Bloomberg LP'

In [ ]:
_search = 'AAL'
tags_autocomplete = requests.get(f'{url}ClipServ/upload/getTagsAutocomplete?sub={_search}&uuid={uuid}', headers=__headers).json()
tags_autocomplete.keys()

In [ ]:
pd.DataFrame(tags_autocomplete['SECURITIES'])

In [ ]:
#####  FIXED INCOME  #####

__cmty = 'Lucas Test CMTY'
__user = 'Lucas Escobar'
__taglist = 'BUYSIDE CREDIT: Research Teams'
__taglist_enum = 'USIG'
__corp = 'AAL Corp'
__security1 = 'AAL 6.5 07/01/2025 Corp'
__security2 = 'AAL 5.9 10/01/2024 Corp'

cmty = NotePublisher.CMTY(**note_pub.buildCMTY(__cmty))
bbg_user_to_share = NotePublisher.BBG_USER(**note_pub.buildUser(user_name=__user,user_type="WORKGROUPS"))
bbg_user_to_tag = NotePublisher.BBG_USER(**note_pub.buildUser(user_name=__user,user_type="PEOPLE"))
bbg_user_author = NotePublisher.BBG_USER(**note_pub.buildUser(user_name=__user,user_type="AUTHORS"))
usig_taglist = NotePublisher.TAGLIST(**note_pub.buildTaglistEnum(__taglist, __taglist_enum))

corp = NotePublisher.NOTE_SECURITY(**note_pub.buildCorpTickerTag(__corp))
security1 = NotePublisher.NOTE_SECURITY(**note_pub.buildSecurityTag(__security1))
security2 = NotePublisher.NOTE_SECURITY(**note_pub.buildSecurityTag(__security2))

In [ ]:
#####  EQUITY  #####

__cmty = 'Lucas Test CMTY'
__user = 'Hamdi Mohamed'
__taglist = 'BUYSIDE EQ: Note Type'
__taglist_enum = 'Company Model'
__security = 'MS US Equity'

cmty = NotePublisher.CMTY(**note_pub.buildCMTY(__cmty))
bbg_user_to_share = NotePublisher.BBG_USER(**note_pub.buildUser(user_name=__user,user_type="WORKGROUPS"))
bbg_user_to_tag = NotePublisher.BBG_USER(**note_pub.buildUser(user_name=__user,user_type="PEOPLE"))
bbg_user_author = NotePublisher.BBG_USER(**note_pub.buildUser(user_name=__user,user_type="AUTHORS"))
model_taglist = NotePublisher.TAGLIST(**note_pub.buildTaglistEnum(__taglist, __taglist_enum))

security = NotePublisher.NOTE_SECURITY(**note_pub.buildSecurityTag(__security))

In [ ]:
# Validating a Pydantic model instance
# try:
#     cmty = NotePublisher.CMTY(**note_pub.buildCMTY(__cmty))
#     print(cmty.dict())  # Serialize model to a dictionary
# except pydantic.ValidationError as e:
#     print(e.json())  # Pretty-print validation errors
#     print(e)

In [ ]:
#####  FIXED INCOME  #####

__note_title = 'API DEMO 5/30/25'
__note_body = """This is an API Test Note!

Tags:
    > People (People and Author)
    > Securities (securities and corp ticker)
    > Taglist(s) (Note Type)

Shareables:
    > CMTYs
    > Individual Users
    > SPDLs (not yet)

Meta Data:
    > Note Title
    > Note Body
    > Note Creation Date

Attachments:
    > Word Document
"""
__as_of_date = int(datetime.datetime.strptime('2025-05-30', '%Y-%m-%d').timestamp() * 1000)
__note_tags = [
    usig_taglist.model_dump(), # Taglists
    bbg_user_to_tag.model_dump(), bbg_user_author.model_dump(), # Users
    corp.model_dump(), security1.model_dump(), security2.model_dump() # Securities
    ]
__share_with = [
    cmty.model_dump(), # CMTYs
    bbg_user_to_share.model_dump() # Users
    ]
__analyst_value = 'Lucas Escobar'
__attachment_files = ['AAL_new_issue_comment.docx']

In [ ]:
#####  EQUITY  #####

__note_title = 'MS API Test Note'
__note_body = """This is an API Test Note!

Tags:
    > People (People and Author)
    > Securities (securities and corp ticker)
    > Taglist(s) (Note Type)

Shareables:
    > CMTYs
    > Individual Users
    > SPDLs (not yet)

Meta Data:
    > Note Title
    > Note Body
    > Note Creation Date

Attachments:
    > Word Document
"""
__as_of_date = int(datetime.datetime.strptime('2025-03-24', '%Y-%m-%d').timestamp() * 1000)
__note_tags = [
    model_taglist.model_dump(), # Taglists
    bbg_user_to_tag.model_dump(), bbg_user_author.model_dump(), # Users
    security.model_dump() # Securities
    ]
__share_with = [
    cmty.model_dump(), # CMTYs
    bbg_user_to_share.model_dump() # Users
    ]
__analyst_value = 'Lucas Escobar'
__attachment_files = ['ms_company_model.xlsx','ms_company_model.pdf']
#__attachment_files = ['RMSBQL.py']

In [ ]:
req = note_pub.buildNoteUploadReq(
    note_title=__note_title,
    note_body=__note_body,
    as_of_date=__as_of_date,
    share_with=__share_with,
    note_tags=__note_tags,
    attachment_file_paths=__attachment_files
)

req

In [ ]:
note_pub.publish_note(req)